In [ ]:
import os

def restart_runtime():
    os.kill(os.getpid(), 9)
    
# !pip3 install --user --extra-index-url https://<username>:<password>@pypi.netsquid.org netsquid
# restart_runtime()

In [ ]:
import netsquid as ns

In [ ]:
from netsquid.components.qsource import QSource, SourceStatus
from netsquid.qubits.state_sampler import StateSampler
from netsquid.components import QuantumChannel
from netsquid.components.models import FixedDelayModel
import netsquid.qubits.ketstates as ks
from netsquid.qubits import qubitapi as qapi


from netsquid.components import QuantumChannel
from netsquid.components.models import DepolarNoiseModel, FibreLossModel
from netsquid.components.models import FibreDelayModel

from netsquid.nodes.connections import Connection
from netsquid.components import ClassicalChannel
from netsquid.components.models import FibreDelayModel

import netsquid.components.instructions as instr
from netsquid.components.qprocessor import QuantumProcessor
from netsquid.components.models.qerrormodels import DephaseNoiseModel, DepolarNoiseModel
from netsquid.components.qprocessor import PhysicalInstruction

In [ ]:
ns.set_qstate_formalism(ns.QFormalism.DM)

# Network Setup

In [ ]:
class ClassicalConnection(Connection):
    def __init__(self, length, name="ClassicalConnection"):
        super().__init__(name=name)
        # forward A Port to ClassicalChannel send Port
        # forward ClassicalChannel recv Port to B Port
        self.add_subcomponent(ClassicalChannel("Channel_A2B", length=length,
                                               models={"delay_model": FibreDelayModel()}),
                              forward_input=[("A", "send")],
                              forward_output=[("B", "recv")])

In [ ]:
class QuantumConnection(Connection):
    def __init__(self, length, depolar_rate, loss_enabled):
        
        super().__init__(name="QuantumConnection")

        if loss_enabled:
          models={"delay_model": FibreDelayModel(),
                    'quantum_noise_model' : DepolarNoiseModel(depolar_rate=depolar_rate),
                    'quantum_loss_model' : FibreLossModel()}
        else:
          models={"delay_model": FibreDelayModel(),
                    'quantum_noise_model' : DepolarNoiseModel(depolar_rate=depolar_rate)}

        self.add_subcomponent(QuantumChannel("qChannel_A2B", length=length,
                              models = models),
                              forward_input=[("A", "send")],
                              forward_output=[("B", "recv")])

In [ ]:
def create_processor(measure_noise_rate, memory_noise_rate):
    """Factory to create a quantum processor.

    Has 2 memory positions and the physical instructions necessary
    for teleportation.

    Parameters
    ----------
    measure_noise_rate : float
        Dephase rate of qubits measurement
    memory_noise_rate : float
        Depolar rate of qubit idling.

    Returns
    -------
    :class:`~netsquid.components.qprocessor.QuantumProcessor`
        A quantum processor to specification.

    """
    measure_noise_model = DephaseNoiseModel(dephase_rate=measure_noise_rate,
                                            time_independent=True)
    physical_instructions = [
        
        PhysicalInstruction(instr.INSTR_INIT, duration=1, parallel=True),

        PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True, topology=[0,1]),

        PhysicalInstruction(instr.INSTR_X, duration=1, parallel=True, topology=[0]),

        PhysicalInstruction(instr.INSTR_Z, duration=1, parallel=True, topology=[0]),

        PhysicalInstruction(instr.INSTR_S, duration=1, parallel=True, topology=[0]),

        PhysicalInstruction(instr.INSTR_CNOT, duration=4, topology=[(0, 1)]),

        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, topology=[0],
                            quantum_noise_model=measure_noise_model),

        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, topology=[1],
                            quantum_noise_model=measure_noise_model)

    ]
    memory_noise_model = DepolarNoiseModel(depolar_rate=memory_noise_rate)

    processor = QuantumProcessor("quantum_processor", num_positions=2,
                                 memory_noise_models=[memory_noise_model]*2,
                                 phys_instructions=physical_instructions)

    return processor

In [ ]:
from netsquid.nodes import Node
from netsquid.nodes import Network

def network_setup(node_distance=4e-3, memory_noise_rate=5e4, measure_noise_rate=0.3, link_noise_rate=0, loss_enabled=True):
    """Setup the physical components of the quantum network.

    Parameters
    ----------
    node_distance : float, optional
        Distance between nodes.
    memory_noise_rate : float, optional
        Depolarization rate of qubits idling in memory.
    measure_noise_rate : float, optional
        Dephasing rate of physical measurement.
    link_noise_rate : float, optional
        Depolarization rate of qubits across channels
    loss_enabled : bool, optional
        Enables photon attenuation across channels

    Returns
    -------
    :class:`~netsquid.nodes.node.Network`

    """
    alice = Node("Alice", qmemory=create_processor(measure_noise_rate, memory_noise_rate))

    bob =  Node("Bob", qmemory=create_processor(measure_noise_rate, memory_noise_rate))

    bp_source = Node("bp_source")

    qsource = QSource(f"qsource_bp_source", StateSampler([ks.b00], [1.0]), num_ports=2,
                          status=SourceStatus.EXTERNAL)
    bp_source.add_subcomponent(qsource, name="qsource")

    network = Network("Quantum Teleportation Network")
    network.add_nodes([alice, bob, bp_source])

    c_conn_a2bps = ClassicalConnection(length=node_distance)

    network.add_connection(alice, bp_source, connection=c_conn_a2bps, label="c_conn_a2bps",
                           port_name_node1="cout_a2bps", port_name_node2="cin_bpsFa")

    bp_source.ports["cin_bpsFa"].forward_input(bp_source.subcomponents["qsource"].ports["trigger"])

    c_conn_a2b = ClassicalConnection(length=node_distance)

    network.add_connection(alice, bob, connection=c_conn_a2b, label="c_conn_a2b",
                           port_name_node1="cout_a2b", port_name_node2="cin_bFa")

    c_conn_b2a = ClassicalConnection(length=node_distance)

    network.add_connection(bob, alice, connection=c_conn_b2a, label="c_conn_b2a",
                           port_name_node1="cout_b2a", port_name_node2="cin_aFb")

    q_conn_bps2a = QuantumConnection(length=node_distance, depolar_rate=link_noise_rate, loss_enabled=loss_enabled)

    qport_bps2a, q_port_aFbps = network.add_connection(bp_source, alice, connection=q_conn_bps2a,
                                                     label="q_conn_bps2a",
                                                     port_name_node1="qout_bps2a",
                                                     port_name_node2="qin_aFbps")

    bp_source.subcomponents["qsource"].ports["qout0"].forward_output(bp_source.ports["qout_bps2a"])

    alice.ports["qin_aFbps"].forward_input(alice.qmemory.ports["qin1"])

    q_conn_bps2b = QuantumConnection(length=node_distance, depolar_rate=link_noise_rate, loss_enabled=False)

    qport_bps2b, q_port_bFbps = network.add_connection(bp_source, bob, connection=q_conn_bps2b,
                                                     label="q_conn_bps2b",
                                                     port_name_node1="qout_bps2b",
                                                     port_name_node2="qin_bFbps")

    bp_source.subcomponents["qsource"].ports["qout1"].forward_output(bp_source.ports["qout_bps2b"])

    bob.ports["qin_bFbps"].forward_input(bob.qmemory.ports["qin0"])

    return network

In [ ]:
from netsquid.components.qprogram import QuantumProgram

class InitQubitXProgram(QuantumProgram):
    """Program to create a qubit and transform it to the y0 state.

    """
    default_num_qubits = 1

    def program(self):
        q1, = self.get_qubit_indices(1)

        self.apply(instr.INSTR_INIT, q1)

        self.apply(instr.INSTR_H, q1)

        self.apply(instr.INSTR_S, q1)

        yield self.run()

In [ ]:
class BellMeasurementProgram(QuantumProgram):
    """Program to perform a Bell measurement on two qubits.

    Measurement results are stored in output keys "M1" and "M2"

    """
    default_num_qubits = 2

    def program(self):
        q1, q2 = self.get_qubit_indices(2)

        self.apply(instr.INSTR_CNOT, [q1, q2])
        self.apply(instr.INSTR_H, q1)
        self.apply(instr.INSTR_MEASURE, q1, output_key="M1")
        self.apply(instr.INSTR_MEASURE, q2, output_key="M2")

        yield self.run()

# Protocols

In [ ]:
from netsquid.protocols import NodeProtocol
from netsquid.protocols import Signals
from netsquid.util.simtools import sim_time
import random

class RequestAndMeasureProtocol(NodeProtocol):
    """Protocol to perform a Bell measurement when qubits are available.

    """

    def __init__(self, node, timeout_window, verbose=False):
      # init Parent NodeProtocol
      super().__init__(node)
      self.verbose = verbose

      self.timeout_window = timeout_window
      self.timeout_count = 0

      self.memory_start_time = None
      self.memory_idle_time = None

    def run(self):
        if self.verbose: print(ns.sim_time(), ": Starting", self.node.name, "s RequestAndMeasureProtocol")

        # ToDo: finish the implementation

        # Initialize state variables and QuantumPrograms
        self.init_program = InitQubitXProgram()
        self.bell_measure_program = BellMeasurementProgram()
        cout_a2bps_port = self.node.ports["cout_a2bps"]
        qin_aFbps_port = self.node.ports["qin_aFbps"]
        cout_a2b_port = self.node.ports["cout_a2b"]
        cin_aFb_port = self.node.ports["cin_aFb"]

        # Initialize qubit X in Alice's memory using InitQubitXProgram.
        yield self.node.qmemory.execute_program(self.init_program, qubit_mapping=[0]) #Protocol waits for the program to finish before going on
        if self.verbose:
            print(ns.sim_time(), "Initialized qubit X in", self.node.name, "'s memory...")
            qubit, = self.node.qmemory.peek(positions=[0])
            print(ns.qubits.reduced_dm(qubit))

        # Track the time qubit X idles in memory
        self.memory_start_time = sim_time()

        while True:
            # Request Bell pair distribution from bp_source
            if self.verbose: print(ns.sim_time(), self.node.name, "'s protocol requests qubit from bp_source")
            cout_a2bps_port.tx_output("REQUEST_QUBIT")
            # Wait for either timeout or input on the quantum port
            recv_expr = yield ((self.await_timer(self.timeout_window)) | (self.await_port_input(qin_aFbps_port)))

            if recv_expr.first_term.value:  # Timeout is true
                self.timeout_count += 1
                if self.verbose: print(ns.sim_time(), self.node.name, "'s protocol timeout_window triggered...")

                if self.timeout_count == 5:
                    # Too many timeouts, end protocol and notify Bob
                    cout_a2b_port.tx_output((-99, -99))
                    if self.verbose:
                        print(f"{ns.sim_time()}: Too many timeouts, ending protocol")
                    break
                else:
                    # Re-request Bell pair
                    if self.verbose:
                        print(ns.sim_time(), "Alice Re-requesting Bell pair\n")
                    continue

            else:  # Bell-state arrived
                if self.verbose: print(ns.sim_time(), self.node.name, "'s protocol received qubit from bp_source")
                if self.verbose: print(self.node.name, f"'s protocol peek at qubit...")
                qubit, = self.node.qmemory.peek(positions=[1])
                print(ns.qubits.reduced_dm(qubit))

                # Perform Bell-state measurement
                self.memory_idle_time = sim_time() - self.memory_start_time
                yield self.node.qmemory.execute_program(self.bell_measure_program, qubit_mapping=[0,1])
                m1 = self.bell_measure_program.output["M1"][0]
                m2 = self.bell_measure_program.output["M2"][0]
                if self.verbose:
                    print(ns.sim_time(), f"Bell-state measurement results: M1={m1}, M2={m2}")

                # Send results to Bob
                cout_a2b_port.tx_output((m1, m2))
                if self.verbose:
                    print(ns.sim_time(), "Outcomes sent to Bob")

                # Wait for Bob's response
                if self.verbose:
                    print(ns.sim_time(), "Waiting for Bob's response")
                yield self.await_port_input(self.node.ports["cin_aFb"])

                bob_message = cin_aFb_port.rx_input().items
                if self.verbose:
                    print(ns.sim_time(), "Bob's notification recieved: ", bob_message)
                break

        self.send_signal(Signals.SUCCESS)

In [ ]:
class ReceiveProtocol(NodeProtocol):
    """Protocol to perform corrections on Bob's qubit when available and measurements received.


    """

    def __init__(self, node, verbose=False):
      # init parent NodeProtocol
      super().__init__(node)

      self.fidelity = None

      self.verbose = verbose

    def run(self):
        if self.verbose: print(ns.sim_time(), ": Starting", self.node.name, "s ReceiveProtocol")

        c_port_bFa = self.node.ports["cin_bFa"] 
        q_port_bFbps = self.node.ports["qin_bFbps"]   
        c_port_b2a = self.node.ports["cout_b2a"]  

        # Wait for classical results from Alice and Bell-state qubit
        recv_expr = yield ((self.await_port_input(c_port_bFa)) & (self.await_port_input(q_port_bFbps)))
        if self.verbose: print(ns.sim_time(), "Bob received qubit from bp_source...")
        qubit, = self.node.qmemory.peek(positions=[0])
        print(ns.qubits.reduced_dm(qubit))

        # Handle classical input from Alice
        alice_message = c_port_bFa.rx_input().items 

        if alice_message[0] == (-99, -99):  # Alice indicates too many timeouts
            if self.verbose:
                print(ns.sim_time(), "Alice experienced too many timeouts. Ending protocol.")
            self.send_signal(Signals.SUCCESS)  # Indicate protocol has ended
            return
        else:
            # Extract Alice's measurements (M1, M2)
            M1, M2 = alice_message[0]
            if self.verbose:
                print(f"{sim_time()}: Received classical measurements from Alice: M1={M1}, M2={M2}")

            # Perform corrections based on Alice's measurements:
            if M1 == 1 and M2 == 1:  # Apply ZX correction
                self.node.qmemory.execute_instruction(instr.INSTR_X, qubit_mapping=[0])
                yield self.await_program(self.node.qmemory)
                self.node.qmemory.execute_instruction(instr.INSTR_Z, qubit_mapping=[0])
                yield self.await_program(self.node.qmemory)
                if self.verbose:
                    print(f"{sim_time()}: Applied X and Z corrections (XZ).")
            elif M1 == 1 and M2 == 0:  # Apply X correction
                self.node.qmemory.execute_instruction(instr.INSTR_X, qubit_mapping=[0])
                yield self.await_program(self.node.qmemory)
                if self.verbose:
                    print(f"{sim_time()}: Applied X correction.")
            elif M1 == 0 and M2 == 1:  # Apply Z correction
                self.node.qmemory.execute_instruction(instr.INSTR_Z, qubit_mapping=[0])
                yield self.await_program(self.node.qmemory)
                if self.verbose:
                    print(f"{sim_time()}: Applied Z correction.")
            else: 
                if self.verbose:
                    print(f"{sim_time()}: No corrections needed.")

            # Measure fidelity of the corrected qubit:
            qubit, = self.node.qmemory.peek(positions=[0])
            print(ns.qubits.reduced_dm(qubit))

            # calculate fidelity
            self.fidelity = qapi.fidelity(qubit, ns.y0, squared=True)
            if self.verbose:
                print(f"{sim_time()}: Calculated fidelity: {self.fidelity}")

            # Notify Alice of success
            c_port_b2a.tx_output("SUCCESS")
            if self.verbose:
                print(f"{sim_time()}: Notified Alice of successful protocol completion.")

            # Indicate protocol has finished
            self.send_signal(Signals.SUCCESS)
            return

# Investigation and Simulation

In [ ]:
from netsquid.util import DataCollector
from netsquid.protocols import Signals
from netsquid.qubits import qubitapi as qapi
import pydynaa

def setup_datacollectors(protocol_alice, protocol_bob):
    """ Collects memory idle time, fidelity, and timeouts,

    Parameters
    ----------
    protocol_alice : :class:`~netsquid.protocols.protocol.Protocol`
        Alice's RequestAndMeasureProtocol().
    protocol_bob : :class:`~netsquid.protocols.protocol.Protocol`
        Bob's ReceiveProtocol().

    Returns
    -------
    :class:`~netsquid.util.datacollector.DataCollector`
        Data collector to record fidelity.

    """

    def get_idle_time(evexpr):
      return {"memory_idle_time": protocol_alice.memory_idle_time}

    def get_fidelity(evexpr):
        return {"fidelity": protocol_bob.fidelity}

    def get_timeouts(evexpr):
      return {"timeouts": protocol_alice.timeout_count}

    dc_mem_idle_time = DataCollector(get_idle_time, include_entity_name=False)
    dc_mem_idle_time.collect_on(pydynaa.EventExpression(source=protocol_alice,
                                          event_type=Signals.SUCCESS.value))

    dc_timeouts = DataCollector(get_timeouts, include_entity_name=False)
    dc_timeouts.collect_on(pydynaa.EventExpression(source=protocol_alice,
                                          event_type=Signals.SUCCESS.value))

    dc_fidelity = DataCollector(get_fidelity, include_entity_name=False)
    dc_fidelity.collect_on(pydynaa.EventExpression(source=protocol_bob,
                                          event_type=Signals.SUCCESS.value))

    return dc_timeouts, dc_mem_idle_time, dc_fidelity

In [ ]:
import pandas

def run_distance_experiment(link_lengths, measure_noise_rate, memory_noise_rate,
                              link_noise_rate, loss_enabled, iterations=10):
    """Run the distance vs. fidelity simulation .

    Parameters
    ----------
    link_lengths : list of float
        List of channel lengths to simulate
    measure_noise_rate : float
        Dephase noise rate of physical measurement
    memory_noise_rate : float
        Depolarization noise rate of qubit idling
    link_noise_rate : float
        Depolarization noise rate of qubits across channels
    loss_enabled : bool
        Enables photon attenuation in fiber
    iterations : int, optional
        Number of simulation iterations to run

    Returns
    -------
    DataFrame tuple (fidelity_data, idle_time_data)
    """
    fidelity_data = pandas.DataFrame()
    timeout_data = pandas.DataFrame()
    idle_time_data = pandas.DataFrame()

    seed = 1

    for link_length in link_lengths:

        for iteration in range(iterations):
            ns.set_random_state(seed=seed)
            seed += 1
            ns.sim_reset()
            network = network_setup(link_length, memory_noise_rate, measure_noise_rate, link_noise_rate, loss_enabled)

            #Protocols
            node_a = network.get_node("Alice")
            node_b = network.get_node("Bob")

            timeout_window = 2 * (link_length / c) + 1
            protocol_alice = RequestAndMeasureProtocol(node_a, timeout_window, verbose=False)
            protocol_bob = ReceiveProtocol(node_b, verbose=False)

            dc_timeouts, dc_mem_idle_time, dc_fidelity = setup_datacollectors(protocol_alice, protocol_bob)

            protocol_alice.start()
            protocol_bob.start()

            ns.sim_run()

            #Save data
            df_fidelity = dc_fidelity.dataframe
            df_fidelity['link_length'] = link_length
            fidelity_data = pandas.concat([fidelity_data, df_fidelity])

            df_timeouts = dc_timeouts.dataframe
            df_timeouts['link_length'] = link_length
            timeout_data = pandas.concat([timeout_data, df_timeouts])

            df_idle_time = dc_mem_idle_time.dataframe
            df_idle_time['link_length'] = link_length
            idle_time_data = pandas.concat([idle_time_data, df_idle_time])

    return fidelity_data, idle_time_data, timeout_data

### No loss simulation

In [ ]:
import numpy

link_lengths = numpy.linspace(0, 50, 50) # km
measure_noise_rate=0
memory_noise_rate=1e4 
link_noise_rate=0 
loss_enabled=False

In [ ]:
from matplotlib import pyplot as plt

def simulate(iterations=100):
    """Show a plot of fidelity versus link_length.
    """
    fidelities, idle_times, timeouts = run_distance_experiment(link_lengths, measure_noise_rate, memory_noise_rate,
                              link_noise_rate, loss_enabled, iterations=iterations)

    plot_style = {'kind': 'scatter', 'grid': True,
                  'title': "Fidelity of the final quantum state"}
    print("Fidelities\n", fidelities)
    print("Idle Times\n", idle_times)
    print("Timeouts\n", timeouts)

    data_fidelity = fidelities.groupby("link_length")['fidelity'].agg(
        fidelity='mean', sem='sem').reset_index()
    print(data_fidelity)
    data_fidelity.plot(x='link_length', y='fidelity', yerr='sem')

    data_idle_time = idle_times.groupby("link_length")['memory_idle_time'].agg(
        memory_idle_time='mean', sem='sem').reset_index()
    print(data_idle_time)
    data_idle_time.plot(x='link_length', y='memory_idle_time', yerr='sem')

    data_timeouts = timeouts.groupby("link_length")['timeouts'].agg(
        timeouts='mean', sem='sem').reset_index()
    print(data_timeouts)
    data_timeouts.plot(x='link_length', y='timeouts', yerr='sem')

    plt.show()

In [ ]:
simulate(iterations=100)

### Loss simulation

In [ ]:
link_lengths = numpy.linspace(0, 50, 50) 
measure_noise_rate=0
memory_noise_rate=1e4 
link_noise_rate=0 
loss_enabled=True

In [ ]:
from matplotlib import pyplot as plt

def simulate(iterations=100):
    """Show a plot of fidelity versus link_length.

    """
    fidelities, idle_times, timeouts = run_distance_experiment(link_lengths, measure_noise_rate, memory_noise_rate,
                              link_noise_rate, loss_enabled, iterations=iterations)

    plot_style = {'kind': 'scatter', 'grid': True,
                  'title': "Fidelity of the final quantum state"}
    data_fidelity = fidelities.groupby("link_length")['fidelity'].agg(
        fidelity='mean', sem='sem').reset_index()
    print(data_fidelity)
    data_fidelity.plot(x='link_length', y='fidelity', yerr='sem')

    data_idle_time = idle_times.groupby("link_length")['memory_idle_time'].agg(
        memory_idle_time='mean', sem='sem').reset_index()
    print(data_idle_time)
    data_idle_time.plot(x='link_length', y='memory_idle_time', yerr='sem')

    data_timeouts = timeouts.groupby("link_length")['timeouts'].agg(
        timeouts='mean', sem='sem').reset_index()
    print(data_timeouts)
    data_timeouts.plot(x='link_length', y='timeouts', yerr='sem')

    plt.show()

In [ ]:
simulate(iterations=100)